In [2]:
import cv2
import numpy as np
import matplotlib.patches as patches
from IPython.display import clear_output
import matplotlib.pylab as plt
import os

try:
    import google.colab
    IN_COLAB = True
    %matplotlib inline
    from google.colab import drive
    from google.colab.patches import cv2_imshow
    drive.mount('/content/gdrive')
    DRIVE_ROOT='/content/gdrive/My Drive/opencv/'
    !pip install face_recognition
except:
    IN_COLAB =False
    %matplotlib notebook  
    DRIVE_ROOT='./'

import face_recognition
face_dir = os.path.join(DRIVE_ROOT,'known_face')
OUT_DIR =os.path.join(DRIVE_ROOT,'output')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
known_face_encodings=[]
known_face_names=[]
subject_image_names=os.listdir(face_dir)
for image_name in subject_image_names:
    if image_name.startswith("."):
        continue
    name=os.path.splitext(image_name)[0];
    image_path = os.path.join(face_dir,image_name)
    image = face_recognition.load_image_file(image_path)
    face_encoding = face_recognition.face_encodings(image)[0]
    known_face_encodings.append(face_encoding)
    known_face_names.append(name)
    print(name)

Asako
ClassMate1
Girl1
ClassMate2
Suzu
Toma


In [0]:
# single frame recognition
def recognize_face(frame,do_scale=False,threshold=0.5,debug=False):
    face_locations=[]
    face_encodings=[]
    face_names=[]
    
    if do_scale:
        # resize frame to 1/4 size for faster processing
        small_frame = cv2.resize(frame, (0,0), fx=0.25, fy=0.25)
    else:
        small_frame = frame
    result=False   
    # convert the image from BGR color to RGB color
    rgb_small_frame = small_frame[:,:,::-1]
    
    # find all the faces and face encodings in the frame
    face_locations = face_recognition.face_locations(rgb_small_frame)
    #if debug:
        #print(face_locations)
    face_encodings = face_recognition.face_encodings(rgb_small_frame,face_locations)
    result=False
    for face_encoding in face_encodings:
        # see if the face is a match for the known face(s)
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        name = 'Unknown'
        
        # use the known face with the smallest distance
        face_distance = face_recognition.face_distance(known_face_encodings,face_encoding)
        best_match_index = np.argmin(face_distance)
        if matches[best_match_index]:
            if face_distance[best_match_index]<threshold:
                name = known_face_names[best_match_index]
            else:
                name = '?'+known_face_names[best_match_index]+'?'
            if debug:
                print(name,face_distance)
        if name == 'Toma':
            result=True
        face_names.append(name)
    
    #display result
    for(top,right,bottom,left),name in zip(face_locations,face_names):
        #scale back up face locations since the frame detected in was scaled to 1/4 size
        if do_scale:
            top *= 4
            right *= 4
            bottom *= 4
            left *= 4
        
        #draw a box around the face
        cv2.rectangle(frame, (left,top), (right,bottom), (0,0,255),2)
        
        # draw a label with a name below the face
        font = cv2.FONT_HERSHEY_PLAIN
        font_scale = 1.2
        thickness = 2
        margin=1
        
        size = cv2.getTextSize(name, font, font_scale, thickness)
        text_width = size[0][0]
        text_height = size[0][1]
        
        cv2.rectangle(frame, (left, bottom), (left+text_width+margin, bottom+text_height+margin), (0, 0, 255), cv2.FILLED)
        cv2.putText(frame, name, (left,bottom + text_height), font,font_scale, (255,255,255), thickness)
    return result,frame

In [87]:
# detect face locations        
def detect_face_locations(image,do_gray=False,do_scale=False,large_frame=False):
    print('detect_face_locations',large_frame)
    if do_scale:
        small_frame = cv2.resize(image, (0,0), fx=0.25, fy=0.25)
    else:
        small_frame = image
    if do_gray:
        gray = cv2.cvtColor(small_frame,cv2.COLOR_BGR2RGB)
        rgb_frame = gray[:,:,::-1]
    else:
        rgb_frame=small_frame[:,:,::-1]

    face_locations = face_recognition.face_locations(rgb_frame)
    for face_location in face_locations:
        (top,right,bottom,left)=face_location;
        if do_scale:
            top *= 4
            right *= 4
            bottom *= 4
            left *= 4

        cv2.rectangle(image, (left,top), (right,bottom), (0,0,255),2)
        if large_frame:
            height, width = image.shape[:2]
            frame_width=int((right-left)/2)
            frame_height=int((bottom-top)/2)
            top=max(0,top-frame_height)
            bottom=min(bottom+frame_height,height)
            left=max(0,left-frame_width)
            right=min(right+frame_width,width)
            print(top,left,bottom,right)
            cv2.rectangle(image, (left,top), (right,bottom), (0,255,255),2)
        #plt.plot(face_location[1],face_location[0],'bo')
        #plt.plot(face_location[1],face_location[2],'bo')
        #plt.plot(face_location[3],face_location[2],'bo')
        #plt.plot(face_location[3],face_location[0],'bo')
        #plt.show()
    return image

In [88]:
# test functions
# format time(second) as HH:MM:ss.mmm
def format_time(sec):
    msec = (int)(sec * 1000.)
    s, ms = divmod(msec, 1000)
    m, s =divmod(s,60)
    return '{:02d}:{:02d}.{:03d}'.format(m,s,ms)

def format_time_forfile(sec):
    msec = (int)(sec * 1000.)
    s, ms = divmod(msec, 1000)
    m, s =divmod(s,60)
    return '{:02d}m{:02d}s{:03d}ms'.format(m,s,ms)

# single image
def single_image(filename,detect_only=False,debug=False,large_frame=False):
#image=cv2.imread("img_interviewMovie.jpg")
    if debug:
        print('single image :',filename,'detect only:',detect_only,'large_frame:',large_frame)
    image=cv2.imread(filename)
    if detect_only:
        image=detect_face_locations(image,large_frame=large_frame)
    else:
        ret,image=recognize_face(image,debug=debug)
    if IN_COLAB:
      cv2_imshow(image)
    else:
      cv2.imshow("sigle test",image)
      if cv2.waitKey(0) & 0xFF:
        cv2.destroyAllWindows()
    
def getInterval(fps):
    interval =1.0/fps
    time_wait = (int)(interval * 1000.0)
    print('interval: ',interval,'time_wait: ',time_wait)
    return interval,time_wait

def video_proc(filename,output=None,capture=False,detect_only=False,no_wait=False,view=True):
    pre = os.path.splitext(os.path.basename(filename))[0]
    cap = cv2.VideoCapture(filename)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    interval,time_wait = getInterval(fps)
    time_stamp=0.0
    frame_count=0
    if no_wait:
        time_wait=1
    
    font = cv2.FONT_HERSHEY_PLAIN
    font_scale = 1.2
    thickness = 2
    margin=1
    size = cv2.getTextSize(name, font, font_scale, thickness)
    text_width = size[0][0]
    text_height = size[0][1]
    counter = 0
    
    # VideoWriter を作成する。
    if output is not None:
        if IN_COLAB:
            fourcc = cv2.VideoWriter_fourcc(*"MJPG")
        else:
            fourcc = cv2.VideoWriter_fourcc(*'DIVX')
        writer = cv2.VideoWriter(os.path.join(OUT_DIR,output), fourcc, fps, (width, height))
    else:
        writer = None
        
    if capture:
      CAPTURE_DIR=os.path.join(OUT_DIR,pre)
      os.makedirs(CAPTURE_DIR,exist_ok=True)

    while cap.isOpened():
        ret,frame = cap.read()
        if not ret:
            cap.release()
            break
        frame_count += 1
        if detect_only:
            frame=detect_face_locations(frame)
        else:
            result,frame=recognize_face(frame)
        cv2.putText(frame,format_time(time_stamp),(margin,margin+text_height),font,font_scale,(0,0,255),thickness)
        if result:
            counter += 1
            print(str(counter),format_time(time_stamp))
            if capture:
                out_name=os.path.join(CAPTURE_DIR,pre+'_'+format_time_forfile(time_stamp)+'.png')
                cv2.imwrite(out_name,frame)
        if not IN_COLAB & view :
            cv2.imshow('video',frame)
            k = cv2.waitKey(time_wait) & 0xff
            if k == 27:
                break;
        if writer is not None:
            writer.write(frame)
        time_stamp += interval
    if not IN_COLAB & view:
        cv2.destroyAllWindows()
    if writer is not None:
        writer.release()
    cap.release()

In [89]:
#image_name="./test-data/IMG_6003.jpg"
#image_name="./test-data/IMG_6002.jpg"
image_name=os.path.join(DRIVE_ROOT,"img_interviewMovie.jpg")
single_image(image_name,detect_only=True,debug=True,large_frame=True)

single image : ./img_interviewMovie.jpg detect only: True large_frame: True
detect_face_locations True
42 256 166 380
35 104 160 228
42 380 166 505
63 477 187 601


In [28]:
import time
# name='gassou_15sec_small'
# name='gassou_full_small'
name='gassou_full_large'
video_name=os.path.join(DRIVE_ROOT,name+'.mp4')
out_name = name+'.avi'
start = time.time()
video_proc(video_name, output=out_name, capture=True, detect_only=False, view=True)
elapsed_time=time.time() - start
print('elapsed time: ',format_time(elapsed_time))

interval:  0.04170833333333333 time_wait:  41
1 00:18.435
2 00:25.900
3 00:42.709
4 00:42.751
5 00:42.792
6 00:42.834
7 00:42.876
8 00:42.917
9 00:42.959
10 00:43.001
11 00:43.042
12 00:43.084
13 00:43.126
14 00:43.168
15 00:43.209
16 00:43.251
17 00:43.293
18 00:43.334
19 00:43.418
20 00:43.460
21 00:43.501
22 01:01.060
23 01:01.186
24 01:36.513
25 01:37.555
26 02:18.304
27 02:18.596
28 02:42.203
29 02:45.248
30 02:45.415
31 02:45.498
32 02:45.540
33 02:45.582
34 02:45.623
35 02:45.665
36 02:45.707
37 02:45.748
38 02:45.915
39 03:13.735
40 03:14.027
41 03:14.068
42 03:14.110
43 03:21.326
44 03:39.761
45 03:42.597
46 03:42.680
47 03:42.764
48 03:49.771
49 03:49.812
50 03:49.854
51 03:50.146
52 03:50.188
53 03:56.235
54 03:56.277
55 04:04.160
56 04:04.452
57 04:04.535
58 04:04.577
59 04:04.619
60 04:04.661
61 04:04.702
62 04:04.744
63 04:04.786
64 04:04.827
65 04:04.869
66 04:04.911
67 04:04.953
68 04:04.994
69 04:05.036
70 04:05.078
71 04:05.119
72 04:05.161
73 04:05.203
74 04:05.244
7

In [30]:
if IN_COLAB:
  file_name = os.path.join(OUT_DIR,out_name)
  from google.colab import files
  files.download(file_name)

----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 53990, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 320, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 351, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 364, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 724, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil